Esempio ottimizzazione a numeri interi per minimizzare il numero di gruppi da creare e il costo ogni 100 ip ---> in realtà mi pare che minimizzi solo per il costo ogni 100 ip, non il numero di gruppi!

In [1]:
import pulp
import json
import math

# 1.0 Data Retrieval

Recupero dati generati da Lu tramite API

In [2]:
res = {"query": "services.http.response.html_title: \"X Acceleration Codec\" or services.http.response.body: \"X Acceleration Codec\"","field": "location.country","total": 2194,"duration": 1099,"total_omitted": 0,"potential_deviation": 0,"buckets": [{"key": "Netherlands","count": 454},{"key": "Spain","count": 392},{"key": "Germany","count": 267},{"key": "France","count": 246},{"key": "United States","count": 139},{"key": "Brazil","count": 108},{"key": "United Kingdom","count": 59},{"key": "Finland","count": 55},{"key": "Bulgaria","count": 41},{"key": "Italy","count": 39},{"key": "Romania","count": 32},{"key": "Ukraine","count": 32},{"key": "Canada","count": 25},{"key": "Serbia","count": 23},{"key": "Bosnia and Herzegovina","count": 21},{"key": "Thailand","count": 19},{"key": "India","count": 18},{"key": "Turkey","count": 16},{"key": "Singapore","count": 15},{"key": "Colombia","count": 14},{"key": "Pakistan","count": 14},{"key": "Poland","count": 14},{"key": "Lithuania","count": 12},{"key": "Israel","count": 10},{"key": "Portugal","count": 10},{"key": "Iraq","count": 9},{"key": "Russia","count": 8},{"key": "Croatia","count": 7},{"key": "Sweden","count": 7},{"key": "Austria","count": 6},{"key": "Switzerland","count": 6},{"key": "Indonesia","count": 5},{"key": "Belgium","count": 4},{"key": "Costa Rica","count": 4},{"key": "Palestinian Territory","count": 4},{"key": "Albania","count": 3},{"key": "Australia","count": 3},{"key": "Czech Republic","count": 3},{"key": "Estonia","count": 3},{"key": "Macedonia","count": 3},{"key": "Peru","count": 3},{"key": "South Africa","count": 3},{"key": "Tunisia","count": 3},{"key": "Argentina","count": 2},{"key": "Cyprus","count": 2},{"key": "Dominican Republic","count": 2},{"key": "Hungary","count": 2},{"key": "Iceland","count": 2},{"key": "Ireland","count": 2},{"key": "Paraguay","count": 2},{"key": "United Arab Emirates","count": 2},{"key": "Armenia","count": 1},{"key": "Bolivia","count": 1},{"key": "Chile","count": 1},{"key": "Denmark","count": 1},{"key": "Ecuador","count": 1},{"key": "Egypt","count": 1},{"key": "El Salvador","count": 1},{"key": "French Polynesia","count": 1},{"key": "Honduras","count": 1},{"key": "Japan","count": 1},{"key": "Jordan","count": 1},{"key": "Malta","count": 1},{"key": "Montenegro","count": 1},{"key": "Morocco","count": 1},{"key": "Myanmar","count": 1},{"key": "Nepal","count": 1},{"key": "Norway","count": 1},{"key": "Slovakia","count": 1},{"key": "Vietnam","count": 1}]}

In [3]:
data = res['buckets']

In [4]:
len(data)

70

In [5]:
data

[{'key': 'Netherlands', 'count': 454},
 {'key': 'Spain', 'count': 392},
 {'key': 'Germany', 'count': 267},
 {'key': 'France', 'count': 246},
 {'key': 'United States', 'count': 139},
 {'key': 'Brazil', 'count': 108},
 {'key': 'United Kingdom', 'count': 59},
 {'key': 'Finland', 'count': 55},
 {'key': 'Bulgaria', 'count': 41},
 {'key': 'Italy', 'count': 39},
 {'key': 'Romania', 'count': 32},
 {'key': 'Ukraine', 'count': 32},
 {'key': 'Canada', 'count': 25},
 {'key': 'Serbia', 'count': 23},
 {'key': 'Bosnia and Herzegovina', 'count': 21},
 {'key': 'Thailand', 'count': 19},
 {'key': 'India', 'count': 18},
 {'key': 'Turkey', 'count': 16},
 {'key': 'Singapore', 'count': 15},
 {'key': 'Colombia', 'count': 14},
 {'key': 'Pakistan', 'count': 14},
 {'key': 'Poland', 'count': 14},
 {'key': 'Lithuania', 'count': 12},
 {'key': 'Israel', 'count': 10},
 {'key': 'Portugal', 'count': 10},
 {'key': 'Iraq', 'count': 9},
 {'key': 'Russia', 'count': 8},
 {'key': 'Croatia', 'count': 7},
 {'key': 'Sweden', 'c

# 2.0 Definizione problema

In [6]:
# Capacità massima per gruppo
capacity = 1000

# Prepara i dati
countries = [item['key'] for item in data]
counts = [item['count'] for item in data]
num_items = len(countries)

# Stima superiore del numero di gruppi (ogni paese in un gruppo separato)
num_bins = num_items

In [7]:
# Creazione del problema
prob = pulp.LpProblem("Bin Packing with API Cost Minimization", pulp.LpMinimize)

# Variabili di decisione
x = pulp.LpVariable.dicts('x', ((i, j) for i in range(num_items) for j in range(num_bins)), cat='Binary')
y = pulp.LpVariable.dicts('y', (j for j in range(num_bins)), cat='Binary')

# Variabili per il costo per gruppo
c = pulp.LpVariable.dicts('c', (j for j in range(num_bins)), lowBound=0, cat='Integer')

# Funzione obiettivo: minimizzare il costo totale
prob += pulp.lpSum(c[j] for j in range(num_bins))

# Vincoli

# Ogni paese deve essere assegnato esattamente a un gruppo
for i in range(num_items):
    prob += pulp.lpSum(x[(i, j)] for j in range(num_bins)) == 1

# La somma dei conteggi in ogni gruppo non deve superare la capacità
for j in range(num_bins):
    prob += pulp.lpSum(counts[i] * x[(i, j)] for i in range(num_items)) <= capacity * y[j]

# Relazione tra c_j e il totale nel gruppo j
for j in range(num_bins):
    total_in_bin = pulp.lpSum(counts[i] * x[(i, j)] for i in range(num_items))
    prob += total_in_bin <= c[j] * 100
    prob += total_in_bin >= c[j] * 100 - 99

# Risoluzione del problema
prob.solve()

/opt/conda/lib/python3.10/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/f5cbdbfaf2a84308ba7e7b20432a99d5-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/f5cbdbfaf2a84308ba7e7b20432a99d5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 285 COLUMNS
At line 30246 RHS
At line 30527 BOUNDS
At line 35568 ENDATA
Problem MODEL has 280 rows, 5040 columns and 19810 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 21.94 - 0.01 seconds
Cgl0004I processed model has 210 rows, 4970 columns (4970 integer (4900 of which binary)) and 14770 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0045I 70 integer variables out of 4970 objects (4970 integer) have cost of 1 - high priority
Cbc0045I branch on satisfied N create fake objective N random cost N
Cbc0038I Initial state - 

1

In [8]:
print("Status:", pulp.LpStatus[prob.status])

# Raccolta dei gruppi utilizzati
bins = {}
for j in range(num_bins):
    if y[j].varValue > 0.5:
        bin_items = []
        total_count = 0
        for i in range(num_items):
            if x[(i, j)].varValue > 0.5:
                bin_items.append((countries[i], counts[i]))
                total_count += counts[i]
        if total_count > 0:
            cost = math.ceil(total_count / 100)
            bins[j] = {'items': bin_items, 'total_count': total_count, 'cost': cost}

Status: Optimal


In [9]:
# Calcolo del costo totale
total_cost = sum(bin_info['cost'] for bin_info in bins.values())

# Numero totale di gruppi utilizzati
total_groups = len(bins)
print(f"Numero totale di gruppi individuati: {total_groups}")
print(f"Costo totale (numero di crediti spesi): {total_cost}")
print("=" * 50)

# Visualizzazione dei gruppi
group_number = 1
for bin_index, bin_info in bins.items():
    print(f"Gruppo {group_number}: Totale = {bin_info['total_count']} | Costo = {bin_info['cost']} crediti")
    for country, count in bin_info['items']:
        print(f" - {country}: {count}")
    print("-" * 50)
    group_number += 1

Numero totale di gruppi individuati: 9
Costo totale (numero di crediti spesi): 22
Gruppo 1: Totale = 398 | Costo = 4 crediti
 - France: 246
 - Brazil: 108
 - Canada: 25
 - Iraq: 9
 - Palestinian Territory: 4
 - Albania: 3
 - Cyprus: 2
 - Jordan: 1
--------------------------------------------------
Gruppo 2: Totale = 100 | Costo = 1 crediti
 - Italy: 39
 - Serbia: 23
 - Colombia: 14
 - Pakistan: 14
 - Austria: 6
 - Hungary: 2
 - Bolivia: 1
 - Montenegro: 1
--------------------------------------------------
Gruppo 3: Totale = 400 | Costo = 4 crediti
 - Spain: 392
 - Croatia: 7
 - Malta: 1
--------------------------------------------------
Gruppo 4: Totale = 100 | Costo = 1 crediti
 - Bulgaria: 41
 - Romania: 32
 - Israel: 10
 - Costa Rica: 4
 - Australia: 3
 - Dominican Republic: 2
 - Paraguay: 2
 - Armenia: 1
 - Egypt: 1
 - El Salvador: 1
 - French Polynesia: 1
 - Nepal: 1
 - Norway: 1
--------------------------------------------------
Gruppo 5: Totale = 100 | Costo = 1 crediti
 - Unite